<a href="https://colab.research.google.com/github/SteelworkerDB/ML_W1/blob/main/Pipelines_Actividad_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Base de Pipelines

---
---

In [24]:
#Carga de librerías

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn import set_config
set_config(display='diagram')

In [4]:
#Carga de datos
filename='/content/Cereal with missing values.xlsx'
df=pd.read_excel(filename)
df.head()

,name,Manufacturer,type,calories per serving,grams of protein,grams of fat,milligrams of sodium,grams of dietary fiber,grams of complex carbohydrates,grams of sugars,milligrams of potassium,vitamins and minerals (% of FDA recommendation),Display shelf,Weight in ounces per one serving,Number of cups in one serving,Rating of cereal
0,Apple Cinnamon Cheerios,General Mills,Cold,110.0,2,2.0,180.0,1.5,10.5,10.0,70,25.0,1,1.00,0.75,29.509541
1,Basic 4,General Mills,Cold,130.0,3,2.0,NaN,2.0,18.0,NaN,100,25.0,3,1.33,0.75,37.038562
2,Cheerios,General Mills,Cold,NaN,6,2.0,290.0,2.0,17.0,1.0,105,25.0,1,1.00,1.25,50.764999
3,Cinnamon Toast Crunch,General Mills,Cold,120.0,1,3.0,210.0,0.0,13.0,9.0,45,25.0,2,1.00,0.75,19.823573
4,Clusters,General Mills,Cold,110.0,3,2.0,140.0,2.0,13.0,7.0,105,25.0,3,1.00,0.50,40.400208


- Definir la característica (X) y objetivo (y)


In [8]:
#La columna a predecir es la de calorías, y la matriz de características son
#el fabricante, tipo de cereal, gramos de grasa, gramos de azúcares
#y el peso en onzas por una porción del cereal.

X=df[['Manufacturer','grams of fat','grams of sugars','Weight in ounces per one serving']]
y=df['calories per serving']
X.head()

,Manufacturer,grams of fat,grams of sugars,Weight in ounces per one serving
0,General Mills,2.0,10.0,1.00
1,General Mills,2.0,NaN,1.33
2,General Mills,2.0,1.0,1.00
3,General Mills,3.0,9.0,1.00
4,General Mills,2.0,7.0,1.00


- Utilizar train test split en los datos para prepararlos para aprendizaje automático.

In [9]:
#Preparación
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

- Identificar cada característica como numérica, ordinal o nominal:
 - *Manufacturer: Nominal*
 - *grams of fat: Numérica*
 - *grams of sugars: Numérica*
 - *Weight in ounces pero one serving: Numérica*
 - *calories per serving: Numérica*

- Utilizar pipelines y transformadores de columnas para completar las siguientes tareas:
 - Imputar algún valor faltante. Utilizar la estrategia “mean” para las columnas numéricas y la estrategia “most_frequent” para las columnas categóricas.
 - Realizar una codificación one-hot a las características nominales.
 - Escalar las columnas numéricas.

In [11]:
#Instanciamiento de selectores de columna
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

In [12]:
#Instanciamiento de los Imputers (para imputación)
freq_imputer = SimpleImputer(strategy='most_frequent')
mean_imputer = SimpleImputer(strategy='mean')

In [13]:
# One-hot encoder para nominales
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)
# Scaler para estandarizar numéricas
scaler = StandardScaler()


- Todos los datos de preprocesamiento deben estar contenidos en un único objeto de preprocesamiento.

In [14]:
# Pipeline numerico
numeric_pipe = make_pipeline(mean_imputer, scaler)
#Pipeline categórico
cat_pipe=make_pipeline(freq_imputer,ohe)

In [18]:
# Tuplas para el Transformador de Columna
number_tuple = (numeric_pipe, num_selector)
cat_tuple = (cat_pipe, cat_selector)
# ColumnTransformer, objeto único de preprocesamiento
preproc = make_column_transformer(number_tuple, cat_tuple)

- Utilizar el objeto de preprocesamiento para transformar los datos de manera adecuada, evitar la fuga de datos para dejarlos listos para el modelado. Muestren los arrays de NumPy resultantes.

In [19]:
# Ajuste del objecto de preproc segúm matriz de entrenamiento
preproc.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x797c01c8bcd0>),
                                ('pipeline-2',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False))]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x797c01c89db0>)])

In [22]:
# Aplicación del objeto sobre la matriz para entrenamiento y prueba
X_train_pro= preproc.transform(X_train)
X_test_pro = preproc.transform(X_test)

In [28]:
#Visualización del numpy array resultante y comprobación
# de la imputación y de la aplicación del OHE


print(np.isnan(X_train_pro).sum().sum(), 'valores faltantes en X_train')
print(np.isnan(X_test_pro).sum().sum(), 'valores faltantes en X_test')
print('\n')
print('Tipo de datos de X_train', X_train_pro.dtype)
print('Tipo de datos de X_test', X_test_pro.dtype)
print('\n')
print('Filas x columnas de X_train', X_train_pro.shape)
print('\n')
X_train_pro

0 valores faltantes en X_train
0 valores faltantes en X_test


Tipo de datos de X_train float64
Tipo de datos de X_test float64


Filas x columnas de X_train (57, 9)




array([[-9.74679434e-01,  9.94481647e-01, -1.32764897e-01,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.22191915e+00,  2.03880702e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [-9.74679434e-01, -8.25018407e-01, -1.32764897e-01,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  1.67679417e+00,  3.15749558e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -1.42705887e-01, -1.32764897e-01,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00, -3.70143393e-01, -1.32764897e-01,
         0.00000000e+00,  0.00000000e+00,  0.000000

Se comprueba que todos los datos son numéricos tipo float.

No hay valores faltantes.


Ref de códigos: PART-TIME DATA SCIENCE - MACHINE LEARNING V22.1